# Allstate Claims Severity - Initial Exploration / データ分析 (EDA)

## 1. Import Libraries / ライブラリのインポート
Import necessary libraries (Pandas, NumPy, Matplotlib, Seaborn).
必要なライブラリを読み込みます。日本語フォント対応もここで行います。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Install and import Japanese font support
# 日本語フォント対応ライブラリのインストールとインポート
!pip install japanize-matplotlib
import japanize_matplotlib

%matplotlib inline
pd.set_option('display.max_columns', None)

## 2. Load Data / データの読み込み
Load the CSV files. Handling paths for both Kaggle and local environment.
CSVファイルを読み込みます。Kaggle環境とローカル環境の両方に対応できるようにパスを自動判定します。

In [ ]:
# Determine the file path / ファイルパスの判定
if os.path.exists('/kaggle/input/allstate-claims-severity/train.csv'):
    base_path = '/kaggle/input/allstate-claims-severity/'
    print('Running on Kaggle (Kaggle環境で実行中)')
else:
    base_path = '../input/'
    print('Running Locally (ローカル環境で実行中)')

train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
sample_submission = pd.read_csv(base_path + 'sample_submission.csv')

print(f'Train shape (学習データ): {train.shape}')
print(f'Test shape (テストデータ): {test.shape}')

## 3. Basic Inspection / 基本的なデータの確認
Check the first few rows and data types (info).
データの先頭行や、型情報(info)を確認します。

In [ ]:
train.head()

In [ ]:
train.info()

## 4. Target Variable Analysis (loss) / 目的変数 (loss) の分析
Analyze the distribution of the target variable `loss`.
予測対象である `loss`（損害額）がどのような分布になっているか確認します。

If the distribution is skewed, log transformation is often applied.
偏りが大きい場合、対数変換（log）をして分布を整えることが一般的です。

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(train['loss'], kde=True, bins=50)
plt.title('Distribution of Loss / 損害額の分布')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(np.log1p(train['loss']), kde=True, bins=50)
plt.title('Distribution of Log(Loss) / 対数変換後の分布')
plt.show()

## 5. Correlation Analysis / 相関分析
Check correlation between continuous variables (`cont1` ~ `cont14`) and `loss`.
連続値の変数 (`cont1` ~ `cont14`) と `loss` の相関を確認します。

High correlation suggests features share similar characteristics.
相関係数が高い変数は、互いに似た性質を持っている可能性があります。

In [ ]:
# List columns containing 'cont' / 列名に 'cont' を含むものをリストアップ
cont_features = [col for col in train.columns if 'cont' in col]
corr = train[cont_features + ['loss']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix (Continuous Features + Loss)\n相関行列（連続値変数 ＋ 損害額）')
plt.show()

## 6. Scatter Plots: Continuous Features vs Loss / 散布図
Visualize the relationship between each continuous feature and `loss`.
各連続値変数と `loss` の関係を散布図で可視化します。

Sampling 1000 points for better visibility.
データ数が多いので、1000件だけランダムに抽出して傾向を見やすくします。

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=2, figsize=(15, 30))

for i, col in enumerate(cont_features):
    row = i // 2
    col_idx = i % 2
    sns.scatterplot(data=train.sample(1000), x=col, y='loss', ax=axes[row, col_idx], alpha=0.5)
    axes[row, col_idx].set_title(f'{col} vs Loss')

plt.tight_layout()
plt.show()

## 7. Categorical Features Analysis / カテゴリ変数の分析
Check the cardinality (number of unique values) of categorical features.
カテゴリ変数 (`cat1` ~ `cat116`) の中身を確認します。
特に「種類の数（ユニーク数）」が多い変数は、扱いが難しいため重要です。

In [ ]:
# List categorical features / カテゴリ変数のリストアップ
cat_features = [col for col in train.columns if 'cat' in col]
print(f'Number of categorical features (カテゴリ変数の数): {len(cat_features)}')

# Count unique values for each feature / 各変数のユニーク数をカウント
unique_counts = train[cat_features].nunique().sort_values(ascending=False)

# Visualize top 30 features with most unique values / 種類が多い上位30個を棒グラフで表示
plt.figure(figsize=(15, 6))
sns.barplot(x=unique_counts.index[:30], y=unique_counts.values[:30])
plt.xticks(rotation=90)
plt.title('Number of Unique Values per Categorical Feature (Top 30)\nカテゴリ変数ごとのユニークな値の数（上位30個）')
plt.show()

# Display top 5 features / 上位5つを表示
print(unique_counts.head(5))

## 8. Analyzing cat116 vs Loss / cat116と損害額の関係
Since `cat116` has the most unique values (high cardinality), let's see how it relates to `loss`.
最も種類が多い `cat116` が、損害額 (`loss`) にどう影響しているか確認します。

We will visualize the distribution of `loss` for the top 10 most frequent categories in `cat116`.
件数が多い上位10個のカテゴリについて、損害額の分布を箱ひげ図（Box Plot）で比較します。

In [ ]:
# Get top 10 categories in cat116 / cat116の中で頻度が高い上位10個を取得
top10_categories = train['cat116'].value_counts().head(10).index

plt.figure(figsize=(15, 8))

# Draw Box Plot / 箱ひげ図の描画
sns.boxplot(x='cat116', y='loss', data=train[train['cat116'].isin(top10_categories)], order=top10_categories)

plt.title('Loss Distribution by cat116 (Top 10 Frequent Categories)\ncat116（頻出上位10カテゴリ）ごとの損害額分布')
plt.yscale('log') # Log scale for better visibility / 金額の桁が違う場合があるので、対数軸で見やすくする
plt.show()